In [76]:
import matrix as m

class BlockInterface:
    def __init__(self, is_collectable, is_movable, is_walkable, name, gameBoard):
        #document the block
        """
        Parameters
        ----------
        is_collectable : bool
            Whether the block can be collected by the player for example keys or diamonds.
        is_movable : bool
            Whether the block can be moved by the player or not.
        is_walkable : bool
            Whether the player can walk on the block or not.
        """
        
        self.is_collectable = is_collectable
        self.is_movable = is_movable
        self.is_walkable = is_walkable 
        self.name = name
        self.gameBoard = gameBoard

    def walkOver(self):
        pass

    def unwalkOver(self):
        pass

    def isReachable(self):
        return False

    def set_is_collectable(self, is_collectable):
        self.is_collectable = is_collectable
    
    def set_is_movable(self, is_movable):
        self.is_movable = is_movable

    def set_is_walkable(self, is_walkable):
        self.is_walkable = is_walkable

    def get_gameBoard(self):
        return self.gameBoard

    def __str__(self):
        return self.name[0:1]

class Wall(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Wall", gameBoard)

class Floor(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Floor", gameBoard)

class Destination(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Destination", gameBoard)

    def isReachable(self):
        #if there is no more diamonds to collect, the destination is reachable
        return super().get_gameBoard().remainingDiamonds == 0

class Key(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(True, False, True, "Key", gameBoard)

    def isReachable(self):
        return True

class Diamond(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(True, False, True, "Diamond", gameBoard)
    
    def isReachable(self):
        return True

class KeyDoor(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "KeyDoor", gameBoard)
    
    def isReachable(self):
        return super().get_gameBoard().currentKeys > 0

class Spikes(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Spikes", gameBoard)

    def walkOver(self):
        pass

class Rock(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, True, True, "Rock", gameBoard)

    def walkOver(self):
        pass

    def isReachable(self):
        return True

class Magma(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Magma", gameBoard)

    def walkOver(self):
        pass

class Hole(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Hole", gameBoard)

    def walkOver(self):
        pass

class Button(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Button", gameBoard)

    def walkOver(self):
        pass

class DoorButton(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "DoorButton", gameBoard)

    def walkOver(self):
        pass

class Character(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Character", gameBoard)

    def walkOver(self):
        pass



class BlockGenerator:
    """
    Class to generate all the objects based on a fixed map
    """
    def __init__(self):
        self.map = {
            'R' : Rock,
            'D' : Diamond,
            'P' : Character,
            'M' : Wall,
            'W' : Magma,
            'A' : Destination,
            'C' : Floor,
            'K' : KeyDoor,
            'L' : Key,
            'S' : Spikes,
            'H' : Hole,
            'B' : Button,
            'U' : DoorButton,
        }

    def generate(self, target, gameBoard):
        """
        returns a new Object based on the given char
        for example:
        'R' -> Rock
        'D' -> Diamond
        """
        return  self.map[target](gameBoard)


class GameBoard:
    def __init__(self):
        self.board = []
        self.currentKeys = 0
        self.remainingDiamonds = 0
        self.characterLocation = (0,0)

    def initialize(self, gameMap, gameBoard):
        self.board = self.generateBoard(gameMap, gameBoard)
        self.generateNumberOfCollectables()

    def generateBoard(self, gameMap, gameBoard):
        """
        Generates the board based on the given map
        """
        board = []
        generator = BlockGenerator()
        for row in gameMap:
            board.append([generator.generate(target, gameBoard) for target in row])
        return board

    def generateNumberOfCollectables(self):
        """
        Iterates over the board and counts the number of diamonds and keys
        """
        for row in self.board:
            for block in row:
                if block.name == "Diamond":
                    self.remainingDiamonds += 1
                if block.name == "Character":
                    self.characterLocation = (self.board.index(row), row.index(block))

    def generateVisitedMatrix(self):
        """
        Generates a matrix of the same size as the board, but filled with False
        """
        return [[False for x in range(len(self.board[0]))] for y in range(len(self.board))]

    def getSpecialBlocksPath(self):
        """
        starting from the character location, get all the speacial blocks for example diamonds, keys, etc
        """

        #list of reachable blocks each element is dict with keys: block, location where location is a list of charters, each character is a location in the board
        reachableBlocks = {} 
        #matrix to keep track of visited blocks
        visited = self.generateVisitedMatrix()
        #seacht the shortest path to each reachable location
        self.getSpecialBlocksRecursive(self.characterLocation, visited, reachableBlocks, [self.characterLocation])

        return reachableBlocks


    def getSpecialBlocksRecursive(self, location, visited, reachableBlocks, path):
        """
        Recursive function to get the shortest path to all the reachable blocks
        """
        curr_row, curr_col = location
        print("current location: " + str(location))

        #if the current block is already visited, return
        if visited[curr_row][curr_col]:
            return
        
        #mark the current block as visited
        visited[curr_row][curr_col] = True

        #get the current block
        block = self.board[curr_row][curr_col]

        #if the current block is not walkable, return
        if not block.is_walkable:
            return
        
        #if the current block is a special block, add it to the reachable blocks
        if block.isReachable():
            if block not in reachableBlocks:
                reachableBlocks[block] = path.copy()
            else:
                if len(path) < len(reachableBlocks[block]):
                    reachableBlocks[block] = path.copy()

        #get the next blocks
        next_blocks = self.getNeighbors(location)

        #if there are no next blocks, return
        if not next_blocks:
            return

        #for each next block, call the function recursively
        for next_block in next_blocks:
            path.append(next_block)
            self.getSpecialBlocksRecursive(next_block, visited, reachableBlocks, path)
            path.pop()

        
    def getNeighbors(self, location):
        """
        returns a list of neighbors of the given location
        """
        neighbors = []
        curr_row, curr_col = location
        for next_row, next_col in [[curr_row + 1, curr_col], [curr_row - 1, curr_col], [curr_row, curr_col + 1], [curr_row, curr_col - 1]]:
            if next_row >= 0 and next_row < len(self.board) and next_col >= 0 and next_col < len(self.board[0]):
                neighbors.append((next_row, next_col))

        return neighbors
    
    def __str__(self) -> str:
        map_str = ""
        for row in self.board:
            for block in row:
                map_str += str(block)
            map_str += "\n"
        return map_str

In [79]:
#test of getting the reachable blocks
mapReachable = [
    ['L', 'C', 'C', 'D'],
    ['M', 'M', 'C', 'C'],
    ['C', 'M', 'C', 'C'],
    ['C', 'C', 'C', 'P'],
]

board = GameBoard()
board.initialize(mapReachable, mapReachable)

print(board.characterLocation == (3,3))
print(board.remainingDiamonds == 1)
print(board.getSpecialBlocksPath())

True
True
current location: (3, 3)
next blocks: [(2, 3), (3, 2)]
current location: (2, 3)
next blocks: [(3, 3), (1, 3), (2, 2)]
current location: (3, 3)
current location: (1, 3)
next blocks: [(2, 3), (0, 3), (1, 2)]
current location: (2, 3)
current location: (0, 3)
next blocks: [(1, 3), (0, 2)]
current location: (1, 3)
current location: (0, 2)
next blocks: [(1, 2), (0, 3), (0, 1)]
current location: (1, 2)
next blocks: [(2, 2), (0, 2), (1, 3), (1, 1)]
current location: (2, 2)
next blocks: [(3, 2), (1, 2), (2, 3), (2, 1)]
current location: (3, 2)
next blocks: [(2, 2), (3, 3), (3, 1)]
current location: (2, 2)
current location: (3, 3)
current location: (3, 1)
next blocks: [(2, 1), (3, 2), (3, 0)]
current location: (2, 1)
current location: (3, 2)
current location: (3, 0)
next blocks: [(2, 0), (3, 1)]
current location: (2, 0)
next blocks: [(3, 0), (1, 0), (2, 1)]
current location: (3, 0)
current location: (1, 0)
current location: (2, 1)
current location: (3, 1)
current location: (1, 2)
curre

In [69]:
#test of generation a signle block
gameBoard = GameBoard()

#TEST 1 for exit block
exitBlock = BlockGenerator().generate('A', gameBoard)

#test to get if the exit is reachable when there are no diamonds
gameBoard.remainingDiamonds = 0
print(exitBlock.isReachable() == True)

#test to get if the exit is reachable when there are diamonds
gameBoard.remainingDiamonds = 1
print(exitBlock.isReachable() == False)

#-------------
#TEST 2 for Door Block
doorBlock = BlockGenerator().generate('K', gameBoard)

gameBoard.currentKeys = 1
print(doorBlock.isReachable() == True)

gameBoard.currentKeys = 0
print(doorBlock.isReachable() == False)

#-------------
#TEST 3 for mapping blocks
mapTest = {}
mapTest[exitBlock] = 1
mapTest[exitBlock] = 3
mapTest[doorBlock] = 2

print(mapTest[exitBlock] == 3)
print(mapTest[doorBlock] == 2)



True
True
True
True
True
True
